In [ ]:
!pip install pyngrok


In [ ]:
!pip install pandas openpyxl deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install flask flask-ngrok xlrd openpyxl deep-translator


In [ ]:
!ngrok authtoken 2q8JNfDfvJ0GybB2c3wkl35VFUD_7B1Pz29pVsvMVXpSMCkL8


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
from openpyxl import load_workbook, Workbook
from deep_translator import GoogleTranslator
from google.colab import files
import xlrd  # Library to handle .xls files

def convert_xls_to_xlsx(xls_file):
    """Convert .xls file to .xlsx using xlrd and openpyxl."""
    wb_xls = xlrd.open_workbook(xls_file)
    wb_xlsx = Workbook()
    sheet_names = wb_xls.sheet_names()

    for sheet_index, sheet_name in enumerate(sheet_names):
        xls_sheet = wb_xls.sheet_by_index(sheet_index)
        xlsx_sheet = wb_xlsx.create_sheet(title=sheet_name if sheet_index > 0 else 'Sheet1')

        # Copy content from .xls to .xlsx sheet
        for row_index in range(xls_sheet.nrows):
            for col_index in range(xls_sheet.ncols):
                xlsx_sheet.cell(row=row_index + 1, column=col_index + 1, value=xls_sheet.cell_value(row_index, col_index))

    xlsx_file = xls_file.replace(".xls", ".xlsx")
    wb_xlsx.save(xlsx_file)
    return xlsx_file

def translate_excel_with_format(input_file, output_file, source_lang="fr", target_lang="en"):
    """Translate text in an Excel file, preserving formatting."""
    # Load workbook
    wb = load_workbook(input_file)
    translator = GoogleTranslator(source=source_lang, target=target_lang)

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Iterate through rows and columns
        for row in ws.iter_rows():
            for cell in row:
                if cell.value and isinstance(cell.value, str):  # Check if cell contains text
                    cell.value = translator.translate(cell.value)

    # Save the translated workbook
    wb.save(output_file)

# Upload multiple files
uploaded = files.upload()

# Process each file
for input_filename in uploaded.keys():
    if input_filename.endswith(".xls"):
        print(f"Converting {input_filename} to .xlsx...")
        input_filename = convert_xls_to_xlsx(input_filename)  # Convert .xls to .xlsx

    if input_filename.endswith(".xlsx"):
        output_filename = f"translated_{input_filename}"  # Create output file name
        translate_excel_with_format(input_filename, output_filename)  # Translate content
        files.download(output_filename)  # Download translated file
    else:
        print(f"Skipping unsupported file: {input_filename}")


Saving mirette.xlsx to mirette.xlsx
Saving NOWEL.xlsx to NOWEL.xlsx
Saving amareen.xlsx to amareen.xlsx
Saving phelopatir.xlsx to phelopatir (1).xlsx
Saving nicholas.xlsx to nicholas.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from flask import Flask, request, send_file
from openpyxl import load_workbook, Workbook
from deep_translator import GoogleTranslator
import xlrd
from pyngrok import ngrok

app = Flask(__name__)

# Function to convert .xls files to .xlsx
def convert_xls_to_xlsx(xls_file):
    wb_xls = xlrd.open_workbook(xls_file)
    wb_xlsx = Workbook()
    sheet_names = wb_xls.sheet_names()
    for sheet_index, sheet_name in enumerate(sheet_names):
        xls_sheet = wb_xls.sheet_by_index(sheet_index)
        xlsx_sheet = wb_xlsx.create_sheet(title=sheet_name if sheet_index > 0 else 'Sheet1')
        for row_index in range(xls_sheet.nrows):
            for col_index in range(xls_sheet.ncols):
                xlsx_sheet.cell(row=row_index + 1, column=col_index + 1, value=xls_sheet.cell_value(row_index, col_index))
    xlsx_file = xls_file.replace(".xls", ".xlsx")
    wb_xlsx.save(xlsx_file)
    return xlsx_file

# Function to translate Excel files
def translate_excel_with_format(input_file, output_file, source_lang="fr", target_lang="en"):
    wb = load_workbook(input_file)
    translator = GoogleTranslator(source=source_lang, target=target_lang)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for row in ws.iter_rows():
            for cell in row:
                if cell.value and isinstance(cell.value, str):
                    cell.value = translator.translate(cell.value)
    wb.save(output_file)

# Flask route to process files
@app.route('/translate', methods=['POST'])
def translate():
    file = request.files['file']
    input_file = file.filename
    file.save(input_file)

    if input_file.endswith('.xls'):
        input_file = convert_xls_to_xlsx(input_file)

    output_file = f"translated_{input_file.replace('.xls', '.xlsx')}"
    translate_excel_with_format(input_file, output_file)

    return send_file(output_file, as_attachment=True)

# Run the Flask app with pyngrok
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)


Public URL: NgrokTunnel: "https://f933-35-247-25-241.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
